In [1]:
import json
import pandas as pd


In [31]:
folder = "../data/"
name = "results.json"
with open(folder + name, "r") as file:
    results = pd.read_json(file, orient="index")

print(results.describe())
results.head()

       predicted changes                      text            timestamp  \
count               2348                      2348                 2348   
unique                 6                      2247                 2291   
top               MEDPOS  make america great again  2016-12-04 22:30:22   
freq                 587                         7                    2   
first                NaN                       NaN  2015-06-14 21:18:23   
last                 NaN                       NaN  2018-09-25 16:45:36   

                            tweet value change  
count                        2348         2348  
unique                       2285            6  
top     MAKE AMERICA GREAT AGAIN!       MEDNEG  
freq                            7          568  
first                         NaN          NaN  
last                          NaN          NaN  


,predicted changes,text,timestamp,tweet,value change
2541,BIGPOS,https https https we all voted for you tonight...,2015-10-29 03:48:02,"""@hyatt1942: @realDonaldTrump @piersmorgan WE ...",MEDNEG
12527,MEDPOS,president donald j trump proclaims october as ...,2017-10-09 18:35:04,"""President Donald J. Trump Proclaims October 9...",MEDPOS
5831,BIGPOS,https https high energy,2016-05-25 05:45:19,"""@buiIdthewall: @realDonaldTrump high energy!""",SMANEG
9855,BIGPOS,record high for s p,2017-09-29 13:39:53,RECORD HIGH FOR S &amp; P 500!,BIGPOS
14647,SMAPOS,just won lawsuit filed by the dnc and a bunch ...,2018-07-06 16:57:34,Just won lawsuit filed by the DNC and a bunch ...,BIGPOS


In [32]:
results = results.reset_index(drop=True)
results.head()

,predicted changes,text,timestamp,tweet,value change
0,BIGPOS,https https https we all voted for you tonight...,2015-10-29 03:48:02,"""@hyatt1942: @realDonaldTrump @piersmorgan WE ...",MEDNEG
1,MEDPOS,president donald j trump proclaims october as ...,2017-10-09 18:35:04,"""President Donald J. Trump Proclaims October 9...",MEDPOS
2,BIGPOS,https https high energy,2016-05-25 05:45:19,"""@buiIdthewall: @realDonaldTrump high energy!""",SMANEG
3,BIGPOS,record high for s p,2017-09-29 13:39:53,RECORD HIGH FOR S &amp; P 500!,BIGPOS
4,SMAPOS,just won lawsuit filed by the dnc and a bunch ...,2018-07-06 16:57:34,Just won lawsuit filed by the DNC and a bunch ...,BIGPOS


In [33]:
neighbours = {"BIGPOS":["MEDPOS"], "MEDPOS":["BIGPOS", "SMAPOS"], "SMAPOS":["MEDPOS", "SMANEG"], "SMANEG":["SMAPOS", "MEDNEG"], "MEDNEG":["SMANEG", "BIGNEG"], "BIGNEG":["MEDNEG"]}
neighbours

{'BIGPOS': ['MEDPOS'],
 'MEDPOS': ['BIGPOS', 'SMAPOS'],
 'SMAPOS': ['MEDPOS', 'SMANEG'],
 'SMANEG': ['SMAPOS', 'MEDNEG'],
 'MEDNEG': ['SMANEG', 'BIGNEG'],
 'BIGNEG': ['MEDNEG']}

In [45]:
rates = {}
rates["correct"] = rates["next"] = rates["wrong"] = 0
wrong = pd.DataFrame(columns=results.columns)

for i, pred in enumerate(results["predicted changes"]):
    actual = results["value change"][i]
    
    if actual == pred:
        rates["correct"] += 1
    elif actual in neighbours[pred]:
        rates["next"] += 1
    else:
        rates["wrong"] += 1
        wrong = wrong.append(results.iloc[i])
    
print(rates)

{'correct': 1039, 'next': 449, 'wrong': 860}


In [46]:
wrong #to take a look which ones we have classified wrong

,predicted changes,text,timestamp,tweet,value change
0,BIGPOS,https https https we all voted for you tonight...,2015-10-29 03:48:02,"""@hyatt1942: @realDonaldTrump @piersmorgan WE ...",MEDNEG
2,BIGPOS,https https high energy,2016-05-25 05:45:19,"""@buiIdthewall: @realDonaldTrump high energy!""",SMANEG
4,SMAPOS,just won lawsuit filed by the dnc and a bunch ...,2018-07-06 16:57:34,Just won lawsuit filed by the DNC and a bunch ...,BIGPOS
7,MEDPOS,watch https on now https,2016-02-05 01:03:23,Watch @AC360 on NOW! @CNN,BIGNEG
8,MEDPOS,https https ia caucus hasn t picked nominee in...,2016-02-06 20:04:09,"""@realOllieTaylor: @paintonmy: IA caucus hasn'...",BIGNEG
9,MEDNEG,https vito thank you mr trump for standing up ...,2016-06-04 12:49:18,"""@Don_Vito_08: Thank You Mr. Trump for Standin...",MEDPOS
12,BIGPOS,sen lindsey graham embarrassed himself with hi...,2016-01-15 15:30:38,Sen. Lindsey Graham embarrassed himself with h...,SMAPOS
16,MEDPOS,rt https report clinton email led to execution...,2016-08-07 21:25:58,RT @DRUDGE_REPORT: CLINTON EMAIL LED TO EXECUT...,SMANEG
19,SMANEG,https https https been watching brilliant mr t...,2015-09-28 00:56:29,"""@truthshallbe: @60Minutes @realDonaldTrump Be...",MEDPOS
22,MEDNEG,the republican house members are working hard ...,2017-11-01 01:22:20,The Republican House members are working hard ...,MEDPOS


In [48]:
#save the success rates

folder = "../data/"
name = "accuracy_rates_from_results.txt"
with open(folder + name, "w") as file:
    file.write(str(rates))

In [52]:
# test if it is readable

with open(folder + name, "r") as file:
    read = literfile.read())

read

ValueError: dictionary update sequence element #0 has length 1; 2 is required